In [1]:
!pip install pulp

In [2]:
import numpy as np
import pandas as pd

In [6]:
data_path='수요지 행정동 분류.csv'
data=pd.read_csv(data_path)
data.head()

FileNotFoundError: [Errno 2] No such file or directory: '수요지 행정동 분류.csv'

In [ ]:
target_path='target_행정동_분류.csv'
target=pd.read_csv(target_path)
target.head()

In [ ]:
dangerous_middle_path='location_middle.csv'
dm=pd.read_csv(dangerous_middle_path, encoding='utf-8')
dangerous_east_path='location_east.csv'
de=pd.read_csv(dangerous_east_path, encoding='utf-8')

In [ ]:
entertainment_df = pd.concat([dm, de], ignore_index=True)
entertainment_df

In [ ]:
# 각 시설물 입지점에서 유흥시설까지의 거리를 계산하는 함수
def is_within_200m_of_entertainment(target_lat, target_lon, entertainment_df):
    for _, row in entertainment_df.iterrows():
        distance = haversine_distance(target_lat, target_lon, row['위도'], row['경도'])
        if distance <= 0.2:  # 200m로 변환
            return True
    return False

In [ ]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # 지구 반지름 (킬로미터)

    # 위도와 경도를 라디안으로 변환
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # 위도와 경도의 차이 계산
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # 허버사인 공식
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = R * c

    return distance

In [ ]:
s=data['초등학령인구'].sum()
s/750

In [ ]:
s/200

##3km 이내

In [ ]:
import pulp
from pulp import LpConstraint

# 데이터 초기화 부분
# ----------------------------------------------
# i의 개수, 즉 수요지의 개수
num_i = data.shape[0]

# j의 개수, 즉 시설물 입지점의 개수
num_j = target.shape[0]

# a_i 값의 리스트. 각 수요지 i의 수요량을 나타냅니다.
a = []
for i in data['초등학령인구']:
  a.append(i)

# b_ij 값의 2차원 리스트. 수요지 i와 시설물 입지점 j 사이의 거리를 나타냅니다.
# 예: b = [[b_11, b_12, ...], [b_21, b_22, ...], ...]
b = []

# 각 수요지에서 모든 시설물까지의 거리를 계산
for i, data_row in data.iterrows():
    distances_for_this_demand = []
    for j, target_row in target.iterrows():
        distance = haversine_distance(data_row['Y'], data_row['X'], target_row['Y'], target_row['X'])
        distances_for_this_demand.append(distance)
    b.append(distances_for_this_demand)

# 설치할 시설물의 개수
# 14개 이상 50개 이하
p = 15
# ----------------------------------------------

# LP 문제 정의. 여기서는 총 거리를 최소화하는 것이 목적입니다.
prob = pulp.LpProblem("Binary_Optimization", pulp.LpMinimize)

# 변수 정의 부분
# ----------------------------------------------
# l_ij는 노드 j의 시설물이 노드 i의 총 수요를 충족하면 1, 아니면 0입니다.
l = pulp.LpVariable.dicts("l", (range(num_i), range(num_j)), 0, 1, pulp.LpBinary)

# k_j는 노드 j에 시설물이 설치되면 1, 아니면 0입니다.
k = pulp.LpVariable.dicts("k", range(num_j), 0, 1, pulp.LpBinary)
# ----------------------------------------------

# 목적함수 정의. 각 수요지와 시설물 입지점 사이의 거리와 해당 수요지의 수요량을 곱한 값을 최소화합니다.
prob += pulp.lpSum([a[i] * b[i][j] * l[i][j] for i in range(num_i) for j in range(num_j)])

# 수동으로 수요지 1을 시설물 입지점 3에 할당합니다.
manually_assigned_facility_for_demand_1 = 2

# 제약조건 설정 부분
# ----------------------------------------------

# 각 수요지 i는 정확히 하나의 시설물 j에만 할당되어야 합니다.
for i in range(1, num_i): # 1번 수요지는 수동 할당되므로 제외(index=0)
    prob += pulp.lpSum([l[i][j] for j in range(num_j)]) == 1

# 설치된 시설물의 총 개수는 p와 동일해야 합니다.
prob += pulp.lpSum([k[j] for j in range(num_j)]) == p

# l_ij는 k_j에 의존적입니다. 즉, k_j가 1일 때만 l_ij는 1이 될 수 있습니다.
for i in range(num_i):
    for j in range(num_j):
        prob += l[i][j] <= k[j]

# 각 시설물 j에 할당된 총 수요량을 계산하고, 750 이하이록 제약 조건을 설정합니다.(도시 지역 폐교 기준 200명을 넘는 결과 도출)
for j in range(num_j):
    total_demand_for_j = pulp.lpSum([a[i] * l[i][j] for i in range(num_i)])
    prob += total_demand_for_j <= 750 * k[j]  # 시설물이 설치되었을 때만 제약이 활성화되어야 합니다.

# 각 거리 b_ij가 3km 이내로만 가능하도록 제약식을 설정합니다.
for i in range(num_i):
    for j in range(num_j):
        prob += l[i][j] <= (3 >= b[i][j])

# 각 시설물 입지점에 대하여 유흥시설과의 거리를 체크하고 제약 조건을 추가
for j, target_row in target.iterrows():
    if is_within_200m_of_entertainment(target_row['Y'], target_row['X'], entertainment_df):
        prob += k[j] == 0


# 시설물 입지점 1, 4, 5, 7, 11, 19 가 할당되지 않도록 하는 제약식 추가
excluded_facility_indices = [0, 3, 4, 6, 10, 18]  # 시설물 입지점 인덱스는 0부터 시작

for j in excluded_facility_indices:
    prob += k[j] == 0
# ----------------------------------------------

# 최적화 문제 풀기
prob.solve()

# 결과 출력 부분
# ----------------------------------------------
print(f"Status: {pulp.LpStatus[prob.status]}")

# 각 수요지에 할당된 시설물 입지점 출력 (값이 1인 경우만 출력)
for i in range(num_i):
    for j in range(num_j):
        if l[i][j].varValue == 1:
            print(f"수요지 {i+1}는 시설물 입지점 {j+1}에 할당됨")

# 설치된 시설물 입지점만 출력 (값이 1인 경우만 출력)
for j in range(num_j):
    if k[j].varValue == 1:
        print(f"시설물 입지점 {j+1}에 설치됨")

# 할당된 시설물 입지점 및 연결된 수요지 정보를 저장할 리스트
assigned_data = []

# 설치된 시설물 입지점 및 배정된 수요량 정보를 저장할 리스트
facility_demand_data = []

# 각 수요지에 할당된 시설물 입지점 정보 추출
for j in range(num_j):
    if k[j].varValue == 1:
        connected_demands = [i+1 for i in range(num_i) if l[i][j].varValue == 1]
        assigned_data.append({'시설물_입지점': j+1, '연결된_수요지': connected_demands})

        total_demand_for_j = sum([a[i-1] for i in connected_demands]) # <- 수정된 부분
        facility_demand_data.append({'시설물_입지점': j+1, '배정된_수요량': total_demand_for_j})

# 데이터프레임으로 변환 후 출력
assigned_df = pd.DataFrame(assigned_data)
print(assigned_df)

facility_demand_df = pd.DataFrame(facility_demand_data)
print(facility_demand_df)

##2.8km 이내

In [ ]:
import pulp
from pulp import LpConstraint

# 데이터 초기화 부분
# ----------------------------------------------
# i의 개수, 즉 수요지의 개수
num_i = data.shape[0]

# j의 개수, 즉 시설물 입지점의 개수
num_j = target.shape[0]

# a_i 값의 리스트. 각 수요지 i의 수요량을 나타냅니다.
a = []
for i in data['초등학령인구']:
  a.append(i)

# b_ij 값의 2차원 리스트. 수요지 i와 시설물 입지점 j 사이의 거리를 나타냅니다.
# 예: b = [[b_11, b_12, ...], [b_21, b_22, ...], ...]
b = []

# 각 수요지에서 모든 시설물까지의 거리를 계산
for i, data_row in data.iterrows():
    distances_for_this_demand = []
    for j, target_row in target.iterrows():
        distance = haversine_distance(data_row['Y'], data_row['X'], target_row['Y'], target_row['X'])
        distances_for_this_demand.append(distance)
    b.append(distances_for_this_demand)

# 설치할 시설물의 개수
# 14개 이상 50개 이하
p = 15
# ----------------------------------------------

# LP 문제 정의. 여기서는 총 거리를 최소화하는 것이 목적입니다.
prob = pulp.LpProblem("Binary_Optimization", pulp.LpMinimize)

# 변수 정의 부분
# ----------------------------------------------
# l_ij는 노드 j의 시설물이 노드 i의 총 수요를 충족하면 1, 아니면 0입니다.
l = pulp.LpVariable.dicts("l", (range(num_i), range(num_j)), 0, 1, pulp.LpBinary)

# k_j는 노드 j에 시설물이 설치되면 1, 아니면 0입니다.
k = pulp.LpVariable.dicts("k", range(num_j), 0, 1, pulp.LpBinary)
# ----------------------------------------------

# 목적함수 정의. 각 수요지와 시설물 입지점 사이의 거리와 해당 수요지의 수요량을 곱한 값을 최소화합니다.
prob += pulp.lpSum([a[i] * b[i][j] * l[i][j] for i in range(num_i) for j in range(num_j)])

# 수동으로 수요지 1을 시설물 입지점 3에 할당합니다.
manually_assigned_facility_for_demand_1 = 2

# 제약조건 설정 부분
# ----------------------------------------------

# 각 수요지 i는 정확히 하나의 시설물 j에만 할당되어야 합니다.
for i in range(1, num_i): # 1번 수요지는 수동 할당되므로 제외(index=0)
    prob += pulp.lpSum([l[i][j] for j in range(num_j)]) == 1

# 설치된 시설물의 총 개수는 p와 동일해야 합니다.
prob += pulp.lpSum([k[j] for j in range(num_j)]) == p

# l_ij는 k_j에 의존적입니다. 즉, k_j가 1일 때만 l_ij는 1이 될 수 있습니다.
for i in range(num_i):
    for j in range(num_j):
        prob += l[i][j] <= k[j]

# 각 시설물 j에 할당된 총 수요량을 계산하고, 750 이하이록 제약 조건을 설정합니다.(도시 지역 폐교 기준 200명을 넘는 결과 도출)
for j in range(num_j):
    total_demand_for_j = pulp.lpSum([a[i] * l[i][j] for i in range(num_i)])
    prob += total_demand_for_j <= 750 * k[j]  # 시설물이 설치되었을 때만 제약이 활성화되어야 합니다.

# 각 거리 b_ij가 2.8km 이내로만 가능하도록 제약식을 설정합니다.
for i in range(num_i):
    for j in range(num_j):
        prob += l[i][j] <= (2.8 >= b[i][j])

# 각 시설물 입지점에 대하여 유흥시설과의 거리를 체크하고 제약 조건을 추가
for j, target_row in target.iterrows():
    if is_within_200m_of_entertainment(target_row['Y'], target_row['X'], entertainment_df):
        prob += k[j] == 0


# 시설물 입지점 1, 4, 5, 7, 11, 19 가 할당되지 않도록 하는 제약식 추가
excluded_facility_indices = [0, 3, 4, 6, 10, 18]  # 시설물 입지점 인덱스는 0부터 시작

for j in excluded_facility_indices:
    prob += k[j] == 0
# ----------------------------------------------

# 최적화 문제 풀기
prob.solve()

# 결과 출력 부분
# ----------------------------------------------
print(f"Status: {pulp.LpStatus[prob.status]}")

# 각 수요지에 할당된 시설물 입지점 출력 (값이 1인 경우만 출력)
for i in range(num_i):
    for j in range(num_j):
        if l[i][j].varValue == 1:
            print(f"수요지 {i+1}는 시설물 입지점 {j+1}에 할당됨")

# 설치된 시설물 입지점만 출력 (값이 1인 경우만 출력)
for j in range(num_j):
    if k[j].varValue == 1:
        print(f"시설물 입지점 {j+1}에 설치됨")

# 할당된 시설물 입지점 및 연결된 수요지 정보를 저장할 리스트
assigned_data = []

# 설치된 시설물 입지점 및 배정된 수요량 정보를 저장할 리스트
facility_demand_data = []

# 각 수요지에 할당된 시설물 입지점 정보 추출
for j in range(num_j):
    if k[j].varValue == 1:
        connected_demands = [i+1 for i in range(num_i) if l[i][j].varValue == 1]
        assigned_data.append({'시설물_입지점': j+1, '연결된_수요지': connected_demands})

        total_demand_for_j = sum([a[i-1] for i in connected_demands]) # <- 수정된 부분
        facility_demand_data.append({'시설물_입지점': j+1, '배정된_수요량': total_demand_for_j})

# 데이터프레임으로 변환 후 출력
assigned_df = pd.DataFrame(assigned_data)
print(assigned_df)

facility_demand_df = pd.DataFrame(facility_demand_data)
print(facility_demand_df)

##2.6km이내

In [ ]:
import pulp
from pulp import LpConstraint

# 데이터 초기화 부분
# ----------------------------------------------
# i의 개수, 즉 수요지의 개수
num_i = data.shape[0]

# j의 개수, 즉 시설물 입지점의 개수
num_j = target.shape[0]

# a_i 값의 리스트. 각 수요지 i의 수요량을 나타냅니다.
a = []
for i in data['초등학령인구']:
  a.append(i)

# b_ij 값의 2차원 리스트. 수요지 i와 시설물 입지점 j 사이의 거리를 나타냅니다.
# 예: b = [[b_11, b_12, ...], [b_21, b_22, ...], ...]
b = []

# 각 수요지에서 모든 시설물까지의 거리를 계산
for i, data_row in data.iterrows():
    distances_for_this_demand = []
    for j, target_row in target.iterrows():
        distance = haversine_distance(data_row['Y'], data_row['X'], target_row['Y'], target_row['X'])
        distances_for_this_demand.append(distance)
    b.append(distances_for_this_demand)

# 설치할 시설물의 개수
# 14개 이상 50개 이하
p = 15
# ----------------------------------------------

# LP 문제 정의. 여기서는 총 거리를 최소화하는 것이 목적입니다.
prob = pulp.LpProblem("Binary_Optimization", pulp.LpMinimize)

# 변수 정의 부분
# ----------------------------------------------
# l_ij는 노드 j의 시설물이 노드 i의 총 수요를 충족하면 1, 아니면 0입니다.
l = pulp.LpVariable.dicts("l", (range(num_i), range(num_j)), 0, 1, pulp.LpBinary)

# k_j는 노드 j에 시설물이 설치되면 1, 아니면 0입니다.
k = pulp.LpVariable.dicts("k", range(num_j), 0, 1, pulp.LpBinary)
# ----------------------------------------------

# 목적함수 정의. 각 수요지와 시설물 입지점 사이의 거리와 해당 수요지의 수요량을 곱한 값을 최소화합니다.
prob += pulp.lpSum([a[i] * b[i][j] * l[i][j] for i in range(num_i) for j in range(num_j)])

# 수동으로 수요지 1을 시설물 입지점 3에 할당합니다.
manually_assigned_facility_for_demand_1 = 2

# 제약조건 설정 부분
# ----------------------------------------------

# 각 수요지 i는 정확히 하나의 시설물 j에만 할당되어야 합니다.
for i in range(1, num_i): # 1번 수요지는 수동 할당되므로 제외(index=0)
    prob += pulp.lpSum([l[i][j] for j in range(num_j)]) == 1

# 설치된 시설물의 총 개수는 p와 동일해야 합니다.
prob += pulp.lpSum([k[j] for j in range(num_j)]) == p

# l_ij는 k_j에 의존적입니다. 즉, k_j가 1일 때만 l_ij는 1이 될 수 있습니다.
for i in range(num_i):
    for j in range(num_j):
        prob += l[i][j] <= k[j]

# 각 시설물 j에 할당된 총 수요량을 계산하고, 750 이하이록 제약 조건을 설정합니다.(도시 지역 폐교 기준 200명을 넘는 결과 도출)
for j in range(num_j):
    total_demand_for_j = pulp.lpSum([a[i] * l[i][j] for i in range(num_i)])
    prob += total_demand_for_j <= 750 * k[j]  # 시설물이 설치되었을 때만 제약이 활성화되어야 합니다.

# 각 거리 b_ij가 2.6km 이내로만 가능하도록 제약식을 설정합니다.
for i in range(num_i):
    for j in range(num_j):
        prob += l[i][j] <= (2.6 >= b[i][j])

# 각 시설물 입지점에 대하여 유흥시설과의 거리를 체크하고 제약 조건을 추가
for j, target_row in target.iterrows():
    if is_within_200m_of_entertainment(target_row['Y'], target_row['X'], entertainment_df):
        prob += k[j] == 0


# 시설물 입지점 1, 4, 5, 7, 11, 19 가 할당되지 않도록 하는 제약식 추가
excluded_facility_indices = [0, 3, 4, 6, 10, 18]  # 시설물 입지점 인덱스는 0부터 시작

for j in excluded_facility_indices:
    prob += k[j] == 0
# ----------------------------------------------

# 최적화 문제 풀기
prob.solve()

# 결과 출력 부분
# ----------------------------------------------
print(f"Status: {pulp.LpStatus[prob.status]}")

# 각 수요지에 할당된 시설물 입지점 출력 (값이 1인 경우만 출력)
for i in range(num_i):
    for j in range(num_j):
        if l[i][j].varValue == 1:
            print(f"수요지 {i+1}는 시설물 입지점 {j+1}에 할당됨")

# 설치된 시설물 입지점만 출력 (값이 1인 경우만 출력)
for j in range(num_j):
    if k[j].varValue == 1:
        print(f"시설물 입지점 {j+1}에 설치됨")

# 할당된 시설물 입지점 및 연결된 수요지 정보를 저장할 리스트
assigned_data = []

# 설치된 시설물 입지점 및 배정된 수요량 정보를 저장할 리스트
facility_demand_data = []

# 각 수요지에 할당된 시설물 입지점 정보 추출
for j in range(num_j):
    if k[j].varValue == 1:
        connected_demands = [i+1 for i in range(num_i) if l[i][j].varValue == 1]
        assigned_data.append({'시설물_입지점': j+1, '연결된_수요지': connected_demands})

        total_demand_for_j = sum([a[i-1] for i in connected_demands]) # <- 수정된 부분
        facility_demand_data.append({'시설물_입지점': j+1, '배정된_수요량': total_demand_for_j})

# 데이터프레임으로 변환 후 출력
assigned_df = pd.DataFrame(assigned_data)
print(assigned_df)

facility_demand_df = pd.DataFrame(facility_demand_data)
print(facility_demand_df)

##2.4km 이내

In [ ]:
import pulp
from pulp import LpConstraint

# 데이터 초기화 부분
# ----------------------------------------------
# i의 개수, 즉 수요지의 개수
num_i = data.shape[0]

# j의 개수, 즉 시설물 입지점의 개수
num_j = target.shape[0]

# a_i 값의 리스트. 각 수요지 i의 수요량을 나타냅니다.
a = []
for i in data['초등학령인구']:
  a.append(i)

# b_ij 값의 2차원 리스트. 수요지 i와 시설물 입지점 j 사이의 거리를 나타냅니다.
# 예: b = [[b_11, b_12, ...], [b_21, b_22, ...], ...]
b = []

# 각 수요지에서 모든 시설물까지의 거리를 계산
for i, data_row in data.iterrows():
    distances_for_this_demand = []
    for j, target_row in target.iterrows():
        distance = haversine_distance(data_row['Y'], data_row['X'], target_row['Y'], target_row['X'])
        distances_for_this_demand.append(distance)
    b.append(distances_for_this_demand)

# 설치할 시설물의 개수
# 14개 이상 50개 이하
p = 15
# ----------------------------------------------

# LP 문제 정의. 여기서는 총 거리를 최소화하는 것이 목적입니다.
prob = pulp.LpProblem("Binary_Optimization", pulp.LpMinimize)

# 변수 정의 부분
# ----------------------------------------------
# l_ij는 노드 j의 시설물이 노드 i의 총 수요를 충족하면 1, 아니면 0입니다.
l = pulp.LpVariable.dicts("l", (range(num_i), range(num_j)), 0, 1, pulp.LpBinary)

# k_j는 노드 j에 시설물이 설치되면 1, 아니면 0입니다.
k = pulp.LpVariable.dicts("k", range(num_j), 0, 1, pulp.LpBinary)
# ----------------------------------------------

# 목적함수 정의. 각 수요지와 시설물 입지점 사이의 거리와 해당 수요지의 수요량을 곱한 값을 최소화합니다.
prob += pulp.lpSum([a[i] * b[i][j] * l[i][j] for i in range(num_i) for j in range(num_j)])

# 수동으로 수요지 1을 시설물 입지점 3에 할당합니다.
manually_assigned_facility_for_demand_1 = 2

# 제약조건 설정 부분
# ----------------------------------------------

# 각 수요지 i는 정확히 하나의 시설물 j에만 할당되어야 합니다.
for i in range(1, num_i): # 1번 수요지는 수동 할당되므로 제외(index=0)
    prob += pulp.lpSum([l[i][j] for j in range(num_j)]) == 1

# 설치된 시설물의 총 개수는 p와 동일해야 합니다.
prob += pulp.lpSum([k[j] for j in range(num_j)]) == p

# l_ij는 k_j에 의존적입니다. 즉, k_j가 1일 때만 l_ij는 1이 될 수 있습니다.
for i in range(num_i):
    for j in range(num_j):
        prob += l[i][j] <= k[j]

# 각 시설물 j에 할당된 총 수요량을 계산하고, 750 이하이록 제약 조건을 설정합니다.(도시 지역 폐교 기준 200명을 넘는 결과 도출)
for j in range(num_j):
    total_demand_for_j = pulp.lpSum([a[i] * l[i][j] for i in range(num_i)])
    prob += total_demand_for_j <= 750 * k[j]  # 시설물이 설치되었을 때만 제약이 활성화되어야 합니다.

# 각 거리 b_ij가 2.4km 이내로만 가능하도록 제약식을 설정합니다.
for i in range(num_i):
    for j in range(num_j):
        prob += l[i][j] <= (2.4 >= b[i][j])

# 각 시설물 입지점에 대하여 유흥시설과의 거리를 체크하고 제약 조건을 추가
for j, target_row in target.iterrows():
    if is_within_200m_of_entertainment(target_row['Y'], target_row['X'], entertainment_df):
        prob += k[j] == 0


# 시설물 입지점 1, 4, 5, 7, 11, 19 가 할당되지 않도록 하는 제약식 추가
excluded_facility_indices = [0, 3, 4, 6, 10, 18]  # 시설물 입지점 인덱스는 0부터 시작

for j in excluded_facility_indices:
    prob += k[j] == 0
# ----------------------------------------------

# 최적화 문제 풀기
prob.solve()

# 결과 출력 부분
# ----------------------------------------------
print(f"Status: {pulp.LpStatus[prob.status]}")

# 각 수요지에 할당된 시설물 입지점 출력 (값이 1인 경우만 출력)
for i in range(num_i):
    for j in range(num_j):
        if l[i][j].varValue == 1:
            print(f"수요지 {i+1}는 시설물 입지점 {j+1}에 할당됨")

# 설치된 시설물 입지점만 출력 (값이 1인 경우만 출력)
for j in range(num_j):
    if k[j].varValue == 1:
        print(f"시설물 입지점 {j+1}에 설치됨")

# 할당된 시설물 입지점 및 연결된 수요지 정보를 저장할 리스트
assigned_data = []

# 설치된 시설물 입지점 및 배정된 수요량 정보를 저장할 리스트
facility_demand_data = []

# 각 수요지에 할당된 시설물 입지점 정보 추출
for j in range(num_j):
    if k[j].varValue == 1:
        connected_demands = [i+1 for i in range(num_i) if l[i][j].varValue == 1]
        assigned_data.append({'시설물_입지점': j+1, '연결된_수요지': connected_demands})

        total_demand_for_j = sum([a[i-1] for i in connected_demands]) # <- 수정된 부분
        facility_demand_data.append({'시설물_입지점': j+1, '배정된_수요량': total_demand_for_j})

# 데이터프레임으로 변환 후 출력
assigned_df = pd.DataFrame(assigned_data)
print(assigned_df)

facility_demand_df = pd.DataFrame(facility_demand_data)
print(facility_demand_df)

##2.3km 이내-infeasible 상태